In [1]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine
from ta import add_all_ta_features


In [2]:
aapl_data = yf.download("AAPL", period="max")


[*********************100%%**********************]  1 of 1 completed


In [37]:
# Convert Date to datetime
aapl_data["Date"] = pd.to_datetime(aapl_data.index)

# Calculate basic indicators
aapl_data["MA9"] = aapl_data["Close"].rolling(window=9).mean()
aapl_data["MA21"] = aapl_data["Close"].rolling(window=21).mean()
aapl_data["MA35"] = aapl_data["Close"].rolling(window=35).mean()
aapl_data["MA50"] = aapl_data["Close"].rolling(window=50).mean()
aapl_data["MA100"] = aapl_data["Close"].rolling(window=100).mean()
aapl_data["MA200"] = aapl_data["Close"].rolling(window=200).mean()

# Select relevant columns
aapl_data = aapl_data[["Date", "Open", "High", "Low", "Close", "Volume", "MA9", "MA21", "MA35", "MA50", "MA100","MA200"]]


C:\Users\Vishnu\AppData\Local\Temp\ipykernel_6408\347775510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl_data["Date"] = pd.to_datetime(aapl_data.index)
C:\Users\Vishnu\AppData\Local\Temp\ipykernel_6408\347775510.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl_data["MA9"] = aapl_data["Close"].rolling(window=9).mean()
C:\Users\Vishnu\AppData\Local\Temp\ipykernel_6408\347775510.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [39]:
# Replace with your database connection details
#only run once to create DB
engine = create_engine("sqlite:///my_stock_data.db")


# Create table if not exists
aapl_data.to_sql("apple_stock", engine, if_exists="replace", index=False)

#sorted descending by date

aapl_data_ordered = pd.read_sql_query("SELECT * FROM apple_stock ORDER BY Date DESC", engine)


# Create table if not exists
aapl_data_ordered.to_sql("apple_stock", engine, if_exists="replace", index=False)



10839

In [35]:

# Example 1: Find all days with RSI above 70 (overbought)
overbought_days = pd.read_sql_query(
    "SELECT * FROM apple_stock WHERE MA21 > 70", engine
)

# Example 2: Calculate average volume for days with MA20 above Close
avg_vol_above_ma20 = pd.read_sql_query(
    "SELECT * FROM apple_stock WHERE MA21 > Close", engine
)


In [36]:
avg_vol_above_ma20

,Date,Open,High,Low,Close,Volume,MA9,MA21,MA35
0,2023-11-01 00:00:00.000000,171.000000,174.229996,170.119995,173.970001,56934900,171.173335,174.787145,174.544287
1,2023-10-31 00:00:00.000000,169.350006,170.899994,167.899994,170.770004,44846000,171.338891,174.712383,174.551144
2,2023-10-30 00:00:00.000000,169.020004,171.169998,168.869995,170.289993,51131000,171.902223,174.854287,174.709144
3,2023-10-27 00:00:00.000000,166.910004,168.960007,166.830002,168.220001,58499100,172.664446,174.898097,174.968287
4,2023-10-26 00:00:00.000000,170.369995,171.380005,165.669998,166.889999,70625300,173.831112,175.015717,175.252859
...,...,...,...,...,...,...,...,...,...
4701,1981-01-20 00:00:00.000000,0.142857,0.142857,0.142299,0.142299,30083200,0.139819,0.143548,NaN
4702,1981-01-16 00:00:00.000000,0.138951,0.138951,0.138393,0.138393,13395200,0.139013,0.140944,NaN
4703,1981-01-15 00:00:00.000000,0.139509,0.140625,0.139509,0.139509,14067200,0.140377,0.139722,NaN
4704,1981-01-14 00:00:00.000000,0.136719,0.137277,0.136719,0.136719,14291200,0.141989,0.138871,NaN


In [7]:
#to kill or dispose the database
engine.dispose()

In [ ]:
#delete table from DB
pd.read_sql_query("DROP TABLE apple_stock", engine)